In [1]:
import pandas as pd
import os
import sys
import time
import pandas as pd
import requests
!pip install meteomatics
!pip install -U airportsdata
import airportsdata
airports = airportsdata.load()  # key is the ICAO identifier (the default)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 1.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.7 MB/s eta 0:00:00


In [3]:
import meteomatics.api as api
import datetime as dt
from pytz import timezone
import pytz


In [15]:
from datetime import datetime
import pytz

from joblib.logger import PrintTime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.compose import make_column_transformer
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor

import numpy as np
from joblib.logger import PrintTime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.compose import make_column_transformer
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier

file_path = 'drive/MyDrive/data/'
resultsDict = {}

for filename in os.listdir(file_path): #iterate through each node in the datafolder
  if filename.endswith(".csv"):
    energyData = pd.read_csv(file_path + filename)

#get data
    res = filename.split("_", 1)
    airport = res[1][:4] # the airport is in the node's name
    
    # get that node's price data
    energyData['Dates'] = pd.to_datetime(energyData['Date/Time']).dt.date
    energyData['Time'] = pd.to_datetime(energyData['Date/Time']).dt.time

    energyData['Dates'] = energyData['Dates'].astype(str)
    energyData["Year"] = energyData['Dates'].str.slice(0,4)
    energyData["Month"] = energyData['Dates'].str.slice(5,7)
    energyData["Day"] = energyData['Dates'].str.slice(8,10)
    energyData["DateAPIFormat"] = energyData.Year + energyData.Month + energyData.Day

    
    #get that node's weatehr data
    weatherjson = requests.get('https://api.synopticlabs.org/v2/stations/timeseries?&token=3066f4e105ff4614a61897b282e96cae&stid='+airport+'&start=202203180000&end=202303182000&timeformat=Y%25YM%25mD%25dH%25HM%25M&vars=air_temp,weather_condition,relative_humidity,wind_speed,wind_direction,wind_gust,solar_radiation,peak_wind_speed,pressure,cloud_layer_1_code')
    weatherData = pd.DataFrame(weatherjson.json()['STATION'][0]["OBSERVATIONS"])
    # weather_df.to_csv("Weather.csv", index=False)
    energyData.to_csv("Energy.csv")
    # weatherData = pd.read_csv("Weather.csv")
    time.sleep(.001)
    energyData2 = pd.read_csv("Energy.csv")


    #get that node's sun data: 
    username = "stanford_chadha"
    password = "TqB5B03dcf"
    coords = [ (str(airports[airport]['lat']),str(airports[airport]['lon']))]
    dates = dt.datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0) - dt.timedelta(days=180)
    end =  dt.datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0) 
    inter =  dt.timedelta(hours=1)
    params = ["direct_rad:W","solar_power:MW"]
    sundf = api.query_time_series(coords, dates, end, inter, params, username, password)


    sundf.to_csv("sun.csv")

    #clean data to local timezone!
    local = pytz.timezone("US/Pacific")
    weatherData.date_time = pd.to_datetime(weatherData.date_time, format='Y%YM%mD%dH%HM%M')
    weatherData.date_time = weatherData.date_time.dt.tz_localize("UTC").dt.tz_convert(local)
    print(energyData2)
    # energyData2.Sunrise = pd.to_datetime(energyData2.Sunrise, format='%Y-%m-%dT%H:%M:%S')
    # energyData2.Sunset = pd.to_datetime(energyData2.Sunset, format='%Y-%m-%dT%H:%M:%S')
    # energyData2.Sunrise = energyData2.Sunrise.dt.tz_convert(local)
    # energyData2.Sunset = energyData2.Sunset.dt.tz_convert(local)
    pd.set_option('display.max_columns', None)
    sun = pd.read_csv("sun.csv")
    sun['validdate']= pd.to_datetime(sun['validdate'])
    sun.validdate = sun.validdate.dt.tz_convert(pytz.timezone("US/Pacific"))
    sun.validdate = pd.to_datetime(sun.validdate, format='Y%YM%mD%dH%HM%M')
    sun.validdate = sun.validdate.dt.tz_convert(local)
    sun["dateJoinFormat"] = sun["validdate"].dt.strftime("%m/%d/%Y %H:%M:%S").str[:19]


    #convert date_time to same style as other one
    weatherData["dateJoinFormat"] = weatherData["date_time"].dt.strftime("%m/%d/%Y %H:%M:%S").str[:19]

    energyData2["Date/Time"]=pd.to_datetime(energyData2['Date/Time'])
    energyData2["dateJoinFormat"]=energyData2["Date/Time"].dt.strftime("%m/%d/%Y %H:%M:%S")
    weatherData.drop_duplicates(subset="dateJoinFormat",
                      keep=False, inplace=True)
    energywithweather = pd.merge(energyData2,weatherData,how='inner',on='dateJoinFormat',validate='one_to_one')

    energyData2["Date/Time"]=pd.to_datetime(energyData2['Date/Time'])
    energyData2["dateJoinFormat"]=energyData2["Date/Time"].dt.strftime("%m/%d/%Y %H:%M:%S")
    weatherData.drop_duplicates(subset="dateJoinFormat",
                      keep=False, inplace=True)
    energywithweather = pd.merge(energyData2,weatherData,how='inner',on='dateJoinFormat',validate='one_to_one')

    sun.drop_duplicates(subset="dateJoinFormat",
                     keep=False, inplace=True)
    
    energywithweatherAndSun = pd.merge(energywithweather,sun,how='inner',on='dateJoinFormat',validate='one_to_one')


    ##DATA ANALYSIS
    data = energywithweatherAndSun
    dataclean = data.iloc[10:,:]
    dataclean.to_csv("dataclean.csv")

    #MACHINE LEARNING
    data['weather_condition_set_1d' ] = data['weather_condition_set_1d'].fillna('') 
    data['cloud_layer_1_code_set_1' ] = data['cloud_layer_1_code_set_1'].fillna(0)
    data['cloud_layer_1_code_set_1' ] = data['cloud_layer_1_code_set_1'].astype(str)
    data['cloud_layer_1_code_set_1'] = data['cloud_layer_1_code_set_1'].str.strip().str[-1]
    

    data['Return $/MW'] = data['Return $/MW'].str.replace(',', '').str.replace('$', '').astype(float)
    data['Return $'] = data['Return $'].str.replace(',', '').str.replace('$', '').astype(float)
    data['Return %'] = data['Return %'].str.replace('%', '').astype(float)/100
    data['Gas Price $/mmBtu'] = data['Gas Price $/mmBtu'].str.replace(',', '').str.replace('$', '').astype(float)
    data['Load '] = data['Load '].str.replace(',', '').str.replace('$', '').astype(float)
    data.to_csv("finaldatabeforemodel.csv")


    X_train, X_test, y_train, y_test = train_test_split(data[["Day","Hour", "Gas Price $/mmBtu", "Load ","direct_rad:W", "solar_power:MW", "Temperature °F",'weather_condition_set_1d',
          'pressure_set_1d', 'relative_humidity_set_1', 'Wind Speed MPH', 'air_temp_set_1',  ]],
                                                        data["Return $/MW"], 
                                                        test_size=0.1, 
                                                        random_state=710)
    #Model Training Code

    # model = KNeighborsClassifier(10)
    model = LGBMRegressor()
    pipeline = make_pipeline(
        make_column_transformer(
            (OneHotEncoder(handle_unknown='ignore'), ["weather_condition_set_1d"]),
            remainder="passthrough"),
            StandardScaler(),
        model)


    resultsDict[filename.split('_')[0]+"mse"] = (-cross_val_score(
        pipeline,
        X_train, y_train,
        scoring="neg_mean_squared_error",
        cv=10).mean())



        #PREDICT BUY SELL
    data["direction"] = np.sign(data["Return $/MW"])

    le.fit(data['direction'])
    data['direction'] = le.fit_transform(data['direction'])

    X_train, X_test, y_train, y_test = train_test_split(data[["Day","Hour", "Gas Price $/mmBtu", "Load ", "direct_rad:W", "solar_power:MW", "Temperature °F",'weather_condition_set_1d', 
          'pressure_set_1d', 'relative_humidity_set_1', 'Wind Speed MPH', 'air_temp_set_1' ]],
                                                        data["direction"], 
                                                        test_size=0.1, 
                                                        random_state=710)
    #Model Training Code

    # model = KNeighborsClassifier(10)
    model = LGBMClassifier()
    pipeline = make_pipeline(
        make_column_transformer(
            (OneHotEncoder(handle_unknown='ignore'), ["weather_condition_set_1d"]),
            remainder="passthrough"),
            StandardScaler(),
        model)

    resultsDict[filename+"accuracy"] = cross_val_score(
        pipeline,
        X_train, y_train,
        scoring="accuracy",
        cv=10).mean()

    pipeline = make_pipeline(
        make_column_transformer(
            (OneHotEncoder(handle_unknown='ignore'), ["weather_condition_set_1d"]),
            remainder="passthrough"),
            StandardScaler(),
        model)

    resultsDict[filename+"f1"] = cross_val_score(
    pipeline,
    X_train, y_train,
    scoring="f1_macro",
    cv=10).mean()

print(resultsDict)


      Unnamed: 0        Date/Time  Day  Hour RT Price $/MW DA Price $/MW  \
0              0   2-6-2023 03:00    6   3.0       -$31.58         $3.80   
1              1   2-6-2023 04:00    6   4.0       -$31.56         $4.43   
2              2   2-6-2023 01:00    6   1.0       -$31.52         $3.85   
3              3   2-6-2023 02:00    6   2.0       -$31.52         $3.86   
4              4   1-2-2023 04:00    2   4.0       -$30.36         $3.71   
...          ...              ...  ...   ...           ...           ...   
8778        8778  7-13-2022 15:00   13  15.0     $3,619.14       $643.09   
8779        8779  5-13-2022 17:00   13  17.0     $4,983.19       $778.19   
8780        8780  7-13-2022 16:00   13  16.0     $5,003.48       $889.35   
8781        8781  7-13-2022 17:00   13  17.0     $5,058.93       $744.36   
8782        8782  7-13-2022 18:00   13  18.0     $5,067.84       $440.05   

     Gas Price $/mmBtu Gas Basis $/mmBtu      Load   Temperature °F  \
0               

<ipython-input-15-c6d362ee059e>:126: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $/MW'] = data['Return $/MW'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-15-c6d362ee059e>:127: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $'] = data['Return $'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-15-c6d362ee059e>:129: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Gas Price $/mmBtu'] = data['Gas Price $/mmBtu'].str.replace(',', '').str.replace(

      Unnamed: 0        Date/Time  Day  Hour RT Price $/MW DA Price $/MW  \
0              0  4-26-2022 11:00   26  11.0       -$37.85        $42.19   
1              1   2-6-2023 03:00    6   3.0       -$31.14         $0.00   
2              2   2-6-2023 01:00    6   1.0       -$31.06        -$0.56   
3              3   2-6-2023 02:00    6   2.0       -$30.99         $0.35   
4              4   2-6-2023 04:00    6   4.0       -$30.97         $0.03   
...          ...              ...  ...   ...           ...           ...   
8778        8778  7-13-2022 15:00   13  15.0     $3,629.86       $639.19   
8779        8779  5-13-2022 17:00   13  17.0     $4,443.74       $569.39   
8780        8780  7-13-2022 16:00   13  16.0     $5,095.42       $888.08   
8781        8781  7-13-2022 17:00   13  17.0     $5,281.03       $749.57   
8782        8782  7-13-2022 18:00   13  18.0     $5,432.49       $448.87   

     Gas Price $/mmBtu Gas Basis $/mmBtu     Load   Temperature °F  \
0                

<ipython-input-15-c6d362ee059e>:126: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $/MW'] = data['Return $/MW'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-15-c6d362ee059e>:127: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $'] = data['Return $'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-15-c6d362ee059e>:129: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Gas Price $/mmBtu'] = data['Gas Price $/mmBtu'].str.replace(',', '').str.replace(

      Unnamed: 0        Date/Time  Day  Hour RT Price $/MW DA Price $/MW  \
0              0  11-4-2022 20:00    4  20.0      -$173.18        $51.11   
1              1   2-6-2023 03:00    6   3.0       -$31.56         $3.11   
2              2   2-6-2023 04:00    6   4.0       -$31.54         $3.62   
3              3   2-6-2023 01:00    6   1.0       -$31.50         $3.05   
4              4   2-6-2023 02:00    6   2.0       -$31.49         $3.16   
...          ...              ...  ...   ...           ...           ...   
8778        8778  7-13-2022 15:00   13  15.0     $3,622.06       $642.13   
8779        8779  5-13-2022 17:00   13  17.0     $4,343.54       $604.30   
8780        8780  7-13-2022 16:00   13  16.0     $5,029.67       $887.70   
8781        8781  7-13-2022 17:00   13  17.0     $5,094.01       $744.46   
8782        8782  7-13-2022 18:00   13  18.0     $5,139.10       $441.05   

     Gas Price $/mmBtu Gas Basis $/mmBtu      Load   Temperature °F  \
0               

...

      Unnamed: 0        Date/Time  Day  Hour RT Price $/MW DA Price $/MW  \
0              0   2-2-2023 08:00    2   8.0       -$32.00        $71.21   
1              1   2-6-2023 03:00    6   3.0       -$31.58         $4.34   
2              2   2-6-2023 04:00    6   4.0       -$31.56         $4.90   
3              3   2-6-2023 01:00    6   1.0       -$31.52         $4.39   
4              4   2-6-2023 02:00    6   2.0       -$31.51         $4.40   
...          ...              ...  ...   ...           ...           ...   
8778        8778  7-13-2022 15:00   13  15.0     $3,617.76       $642.76   
8779        8779  7-13-2022 16:00   13  16.0     $4,992.21       $885.57   
8780        8780  5-13-2022 17:00   13  17.0     $5,022.68       $817.58   
8781        8781  7-13-2022 18:00   13  18.0     $5,030.50       $433.64   
8782        8782  7-13-2022 17:00   13  17.0     $5,034.38       $738.32   

     Gas Price $/mmBtu Gas Basis $/mmBtu      Load   Temperature °F  \
0               

<ipython-input-15-c6d362ee059e>:126: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $/MW'] = data['Return $/MW'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-15-c6d362ee059e>:127: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $'] = data['Return $'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-15-c6d362ee059e>:129: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Gas Price $/mmBtu'] = data['Gas Price $/mmBtu'].str.replace(',', '').str.replace(

      Unnamed: 0        Date/Time  Day  Hour RT Price $/MW DA Price $/MW  \
0              0  3-18-2022 05:00   18   5.0       -$49.96        -$4.15   
1              1  3-18-2022 04:00   18   4.0       -$35.69        -$5.35   
2              2  4-17-2022 21:00   17  21.0       -$33.07        $82.76   
3              3   2-6-2023 02:00    6   2.0       -$32.11        -$2.33   
4              4   2-6-2023 01:00    6   1.0       -$32.08        -$3.20   
...          ...              ...  ...   ...           ...           ...   
8778        8778  7-13-2022 15:00   13  15.0     $3,629.79       $672.22   
8779        8779  5-13-2022 17:00   13  17.0     $4,511.76       $600.37   
8780        8780  7-13-2022 16:00   13  16.0     $5,094.77       $921.58   
8781        8781  7-13-2022 17:00   13  17.0     $5,278.86       $780.64   
8782        8782  7-13-2022 18:00   13  18.0     $5,427.58       $478.40   

     Gas Price $/mmBtu Gas Basis $/mmBtu     Load   Temperature °F  \
0                

<ipython-input-15-c6d362ee059e>:126: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $/MW'] = data['Return $/MW'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-15-c6d362ee059e>:127: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Return $'] = data['Return $'].str.replace(',', '').str.replace('$', '').astype(float)
<ipython-input-15-c6d362ee059e>:129: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Gas Price $/mmBtu'] = data['Gas Price $/mmBtu'].str.replace(',', '').str.replace(

{'HBHoustonmse': 4602.752528265027, 'HBHouston_KIAH.csvaccuracy': 0.8041770010465801, 'HBHouston_KIAH.csvf1': 0.7496441407549207, 'HBWestmse': 8227.41759046345, 'HBWest_KSJT.csvaccuracy': 0.8037433155080216, 'HBWest_KSJT.csvf1': 0.7626976434228324, 'HBSouthmse': 6629.731634011007, 'HBSouth_KSJT.csvaccuracy': 0.8136363636363637, 'HBSouth_KSJT.csvf1': 0.730902546104714, 'HBNorthmse': 6747.773465769875, 'HBNorth_KDAL.csvaccuracy': 0.8108303222367018, 'HBNorth_KDAL.csvf1': 0.7562168030961297, 'TOPAZALLmse': 4559.313992988422, 'TOPAZALL_KGLS.csvaccuracy': 0.8005517251875865, 'TOPAZALL_KGLS.csvf1': 0.7588411934709833, 'BRAZWINDmse': 7933.7583521407605, 'BRAZWIND_KLBB.csvaccuracy': 0.8039263166998183, 'BRAZWIND_KLBB.csvf1': 0.7102530673066239}


# As you can see from the final results dictionary values of our accuracy (if you scroll to the bottom!)
Node #1: 
Houston MSE: 462 (not bad!) 
Houston Accuracy: 0.80
Houston F1 = 0.75

Node #2: 
HBWest Accuracy: 0.80
HBWest F1: 0.76

Node #3: 
HBSouth Accuracy: 0.81
HBSouth F1: 0.73

Node #4
HBNorth Accuracy: 0.81
HBNorth F1: 0.76

Node #5
TopazAll Accuracy: 0.80
TOPAZALL F1: 0.76

Node #6: 
BrazWind Accuracy: 0.80
Brazwind f1: 0.71

# **Which is pretty good!!!!!**

My next steps involve including more data on transmission line outages, generator outages, cloud cover, and I think it will get even better! 